<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [14]:
# the package we'll use to send an 
# HTTP request to Hugging Face's API import requests
import requests
# A URL to indicate which model we'll use. 
#If you visit it, the page displays information on the model.
API_URL = "https://api-inference.huggingface.co/models/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis"
# A dictionary that contains our private key.
# Be sure change this to include your own key. 
headers = {"private": "false"}
#{"do_lower_case": true, "unk_token": "[UNK]", "sep_token": "[SEP]", "pad_token": "[PAD]", "cls_token": "[CLS]", "mask_token": "[MASK]", "tokenize_chinese_chars": true, "strip_accents": null, "model_max_length": 512, "name_or_path": "bert-base-uncased"}
# A method that we'll call to run the model 
def query(payload):
 # Makes a request to Hugging Face's API  
 response = requests.post(API_URL, headers=headers, json=payload)
 return response.json()

In [15]:
output = query("Tesla's stock decreased by 20% today!")
print(output)

[[{'label': 'negative', 'score': 0.9984173774719238}, {'label': 'neutral', 'score': 0.0010758034186437726}, {'label': 'positive', 'score': 0.0005068433238193393}]]


In [16]:
output = query("Tesla's stock rocketed by 20% today!")
print(output)

[[{'label': 'negative', 'score': 0.00022498672478832304}, {'label': 'neutral', 'score': 0.00017525360453873873}, {'label': 'positive', 'score': 0.9995997548103333}]]


In [59]:
article = 'Stocks continue their sell-off, with Tesla, Amazon, and Alibaba among the biggest decliners.Video Transcript[MUSIC PLAYING]- A massive sell-off with just over 25 minutes-- 35 minutes to go in the trading day. Dow still off just over 1,300 points. S&P and NASDAQ firmly in the red.Taking a look at some of the trending tickers on our site. The biggest decliners that we are seeing. Tesla off just nearly 10%. Amazon, Alibaba, Aurora, and Apple rounding out the top five trending tickers, biggest decliners, on our site.'

In [34]:
from transformers import pipeline
roberta = pipeline(task='sentiment-analysis', 
                   model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis",
                   return_all_scores=True,
                   max_length=514, 
                   truncation=True)

In [60]:
# yahoo web scrape NLP

    
score_df = pd.DataFrame()
#     sentence_df = pd.DataFrame()
#     tag_df = pd.DataFrame(columns=["tags"])
    
#     nf = "Content is currently unavailable"
    
# #for i in range(len(urls)): #filter_urls
#     #URL = #urls.iloc[i] # filter_urls
#     page = requests.get(URL,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
#     soup = BeautifulSoup(page.content, "html.parser")
#     if nf in soup.prettify():
#         text = "NA"
#         score_df = score_df.append(read_grades(text))
#         sentence_df = sentence_df.append(sentence_info(text))
#         tag_df.at[i, 'tags'] = tag_article(text,companies,tick_comp_df)

#     try:
#         try:
#             master = soup.find(id="Masterwrap")
#             final = master.find("div", class_="caas-body")
#         except AttributeError:
#             master = soup.find(id="Masterwrap2Col")
#             final = master.find("div", class_="caas-body")
#     except AttributeError:
#         pass

#     text = re.sub('(<p>)', ' ', str(final))
#     text = re.sub('<[^>]+>', '', str(text))
sen_list = article.split(". ")
article = [".\n ".join(sen_list[:-2])][0] # remove author signature
#score_df = score_df.append(read_grades(article))
#sentence_df = sentence_df.append(sentence_info(article))
#tag_df.at[i, 'tags'] = tag_article(article,companies,tick_comp_df)

pos = []
neg = []
neu = []
net = []

for sen in sen_list:
    output = roberta(sen)
    #output = roberta(sen[0:514])
    neg.append(output[0][0]['score'])
    neu.append(output[0][1]['score'])
    pos.append(output[0][2]['score'])
    net.append(pos[-1] - neg[-1])

#     df.pos_mean.iloc[i], df.pos_median.iloc[i], df.pos_min.iloc[i], df.pos_max.iloc[i] = summary_stats(pos)
#     df.neg_mean.iloc[i], df.neg_median.iloc[i], df.neg_min.iloc[i], df.neg_max.iloc[i] = summary_stats(neg)
#     df.neu_mean.iloc[i], df.neu_median.iloc[i], df.neu_min.iloc[i], df.neu_max.iloc[i] = summary_stats(neu)
#     df.net_mean.iloc[i], df.net_median.iloc[i], df.net_min.iloc[i], df.net_max.iloc[i] = summary_stats(net)

#     score_df.index = df.index
#     sentence_df.index = df.index
#     tag_df.index = df.index
#     df = pd.concat([df,score_df,sentence_df,tag_df],axis=1)
#     df = df.dropna()
        

In [63]:
sen_list

['Stocks continue their sell-off, with Tesla, Amazon, and Alibaba among the biggest decliners.Video Transcript[MUSIC PLAYING]- A massive sell-off with just over 25 minutes-- 35 minutes to go in the trading day',
 'Dow still off just over 1,300 points',
 'S&P and NASDAQ firmly in the red.Taking a look at some of the trending tickers on our site',
 'The biggest decliners that we are seeing',
 'Tesla off just nearly 10%',
 'Amazon, Alibaba, Aurora, and Apple rounding out the top five trending tickers, biggest decliners, on our site.']

In [62]:
print(pos)

[0.000609890092164278, 0.0009496893617324531, 0.00010529914288781583, 6.586547533515841e-05, 0.0005465730791911483, 6.093690171837807e-05]


In [71]:
df = pd.DataFrame(columns={"sentence", "positive", "neutral", "negative", "net"})
df.sentence=sen_list
df.positive=pos
df.neutral=neu
df.negative=neg
df.net=net
df.to_excel("sentence_scores.xlsx", index=False)

In [72]:
df

,sentence,negative,neutral,positive,net
0,"Stocks continue their sell-off, with Tesla, Am...",0.992247,0.007143,0.000610,-0.991637
1,"Dow still off just over 1,300 points",0.996589,0.002462,0.000950,-0.995639
2,S&P and NASDAQ firmly in the red.Taking a look...,0.000083,0.999812,0.000105,0.000022
3,The biggest decliners that we are seeing,0.000141,0.999793,0.000066,-0.000075
4,Tesla off just nearly 10%,0.997608,0.001846,0.000547,-0.997061
5,"Amazon, Alibaba, Aurora, and Apple rounding ou...",0.000068,0.999871,0.000061,-0.000007
